In [1]:
import os
import numpy as np
from matplotlib import pyplot
# Modulo de optimización
from scipy import optimize
%matplotlib inline

Marca ,Número de puertas, Número de asientos, Condición del vehículo,Transmisión,Tipo de Combustible,Tipo,"Modelo (Ej: 2021, 2016)",Procedencia,Kilometraje (recorrido en Km),Cilindrada (en cc),Estado de la pintura,¿La pintura está en buen estado?,"¿El vehículo tuvo algún accidente? ","¿El vehículo cuenta con todos los accesorios? ",Costo en $us,

In [30]:
data_train = np.loadtxt('preparado.csv', delimiter=',', dtype=float)
data_test = np.loadtxt('test.csv', delimiter=',', dtype=float)
#print(data_train)
#print(data_test)


In [31]:
X = data_train[:,0:15]
X_test = data_test[:,0:15]
#print(X_test)

In [32]:
y = data_train[:,15]
y_test = data_test[:,15]
#y_test.shape,y.shape , y_test
m, m_test= y.size ,y_test.size
m

835

In [33]:
X = np.concatenate([np.ones((m, 1)), X], axis=1)
X_test = np.concatenate([np.ones((m_test, 1)), X_test], axis=1)


In [36]:
#X_test

In [37]:
def normalEqn(X, y):
    theta = np.zeros(X.shape[1])
    theta = np.dot(np.dot(np.linalg.inv(np.dot(X.T,X)),X.T),y)
    return theta

In [38]:
theta = normalEqn(X, y);
# Muestra los resultados optenidos a partir de la aplicación de la ecuación de la normal
print('Theta calculado a partir de la ecuación de la normal: {:s}'.format(str(theta)));

Theta calculado a partir de la ecuación de la normal: [-5.94619377e+05 -1.17266071e+02 -1.27651242e+01  6.21360133e+02
 -2.89827416e+03 -2.54590551e+03 -4.21942366e+03  3.56600789e+03
  3.09723019e+02 -1.92247365e+02 -6.72445916e-04  6.07502094e-01
 -2.79657216e+02 -1.43083103e+03  1.02609883e+03 -1.30502577e+03]


In [39]:
prices=[]
for x_t in X_test:
    prices.append(int(np.dot(x_t, theta)))
print('precios Predecidos (usando la ecuación de la normal): $')
print(prices)

precios Predecidos (usando la ecuación de la normal): $
[33338, 36980, 25582, 21062, 9355, 17446, 16678]


**usando una Red neuronal** 

In [40]:
#funciondes de activacion
def sigmoid(z):
    """
    Computes the sigmoid of z.
    """
    return 1.0 / (1.0 + np.exp(-z))

def relu(x):
  return np.maximum(0, x)

def reluPrime(x):
  return x > 0

In [41]:
def linear(x):
    return x

def sigmoid(x):
  return 1 / (1 + np.exp(-x))

def softmax(x):
    return np.exp(x) / np.exp(x).sum(axis=-1,keepdims=True)

funciones de perdida

In [42]:
# Mean Square Error -> usada para regresión (con activación lineal)
def mse(y, y_hat):
    return np.mean((y_hat - y.reshape(y_hat.shape))**2)

# Binary Cross Entropy -> usada para clasificación binaria (con sigmoid)
def bce(y, y_hat):
    return - np.mean(y.reshape(y_hat.shape)*np.log(y_hat) - (1 - y.reshape(y_hat.shape))*np.log(1 - y_hat))

# Cross Entropy (aplica softmax + cross entropy de manera estable) -> usada para clasificación multiclase
def crossentropy(y, y_hat):
    logits = y_hat[np.arange(len(y_hat)),y]
    entropy = - logits + np.log(np.sum(np.exp(y_hat),axis=-1))
    return entropy.mean()

Derivadas

In [43]:
def grad_mse(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)

def grad_bce(y, y_hat):
    return y_hat - y.reshape(y_hat.shape)

def grad_crossentropy(y, y_hat):
    answers = np.zeros_like(y_hat)
    answers[np.arange(len(y_hat)),y] = 1    
    return (- answers + softmax(y_hat)) / y_hat.shape[0]

In [44]:
# clase base MLP 

class MLP():
  def __init__(self, D_in, H, D_out, loss, grad_loss, activation):
    # pesos de la capa 1
    self.w1, self.b1 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(D_in+H)),
                                  size=(D_in, H)), np.zeros(H)
    # pesos de la capa 2
    self.w2, self.b2 = np.random.normal(loc=0.0,
                                  scale=np.sqrt(2/(H+D_out)),
                                  size=(H, D_out)), np.zeros(D_out)
    self.ws = []
    # función de pérdida y derivada
    self.loss = loss
    self.grad_loss = grad_loss
    # función de activación
    self.activation = activation

  def __call__(self, x):
    # salida de la capa 1
    self.h_pre = np.dot(x, self.w1) + self.b1
    self.h = relu(self.h_pre)
    # salida del MLP
    y_hat = np.dot(self.h, self.w2) + self.b2 
    return self.activation(y_hat)
    
  def fit(self, X, Y, epochs = 100, lr = 0.001, batch_size=None, verbose=True, log_each=1):
    batch_size = len(X) if batch_size == None else batch_size
    batches = len(X) // batch_size
    l = []
    for e in range(1,epochs+1):     
        # Mini-Batch Gradient Descent
        _l = []
        for b in range(batches):
            # batch de datos
            x = X[b*batch_size:(b+1)*batch_size]
            y = Y[b*batch_size:(b+1)*batch_size] 
            # salida del perceptrón
            y_pred = self(x) 
            # función de pérdida
            loss = self.loss(y, y_pred)
            _l.append(loss)        
            # Backprop 
            dldy = self.grad_loss(y, y_pred) 
            grad_w2 = np.dot(self.h.T, dldy)
            grad_b2 = dldy.mean(axis=0)
            dldh = np.dot(dldy, self.w2.T)*reluPrime(self.h_pre)      
            grad_w1 = np.dot(x.T, dldh)
            grad_b1 = dldh.mean(axis=0)
            # Update (GD)
            self.w1 = self.w1 - lr * grad_w1
            self.b1 = self.b1 - lr * grad_b1
            self.w2 = self.w2 - lr * grad_w2
            self.b2 = self.b2 - lr * grad_b2
        l.append(np.mean(_l))
        # guardamos pesos intermedios para visualización
        self.ws.append((
            self.w1.copy(),
            self.b1.copy(),
            self.w2.copy(),
            self.b2.copy()
        ))
        if verbose and not e % log_each:
            print(f'Epoch: {e}/{epochs}, Loss: {np.mean(l):.5f}')

  def predict(self, ws, x):
    w1, b1, w2, b2 = ws
    h = relu(np.dot(x, w1) + b1)
    y_hat = np.dot(h, w2) + b2
    return self.activation(y_hat)

In [78]:
# MLP para regresión
class MLPRegression(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, mse, grad_mse, linear)

# MLP para clasificación binaria
class MLPBinaryClassification(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, bce, grad_bce, sigmoid)

# MLP para clasificación multiclase
class MLPClassification(MLP):
    def __init__(self, D_in, H, D_out):
        super().__init__(D_in, H, D_out, crossentropy, grad_crossentropy, linear)

cargamos los datos de entrenamiento y de prueba


In [80]:
X = data_train[:,0:15]
X_test = data_test[:,0:15]

y = data_train[:,15]
y_test = data_test[:,15]


In [81]:
# normalización datos
X_mean = X.mean(axis=0)
X_std = X.std(axis=0)
X_norm = (X - X_mean) / X_std

In [82]:
X_norm.shape

(835, 15)

In [87]:
model = MLPRegression(D_in=15, H=10, D_out=1)
epochs, lr = 10, 0.0001
model.fit(X_norm, y, epochs, lr, batch_size=1, log_each=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:48: RuntimeWarning: invalid value encountered in multiply


Epoch: 1/10, Loss: nan
Epoch: 2/10, Loss: nan
Epoch: 3/10, Loss: nan
Epoch: 4/10, Loss: nan
Epoch: 5/10, Loss: nan
Epoch: 6/10, Loss: nan
Epoch: 7/10, Loss: nan
Epoch: 8/10, Loss: nan
Epoch: 9/10, Loss: nan
Epoch: 10/10, Loss: nan
